In [1]:
import pybullet as p
import pybullet_data
import os
import numpy as np
import time
physicsClient = p.connect(p.GUI)
# physicsClient = p.connect(p.GUI_SERVER) # does tno help

# physicsClient = p.connect(p.GUI, options='--opengl2') # the graphics problem does not happen here! but its ugly.... 

In [ ]:
p.resetSimulation() # remove all objects from the world and reset the world to initial conditions. (not needed here but kept for example)
p.configureDebugVisualizer(p.COV_ENABLE_GUI,0,physicsClientId=physicsClient)

p.setGravity(0,0,-9.81)
planeId = p.loadURDF(os.path.join(pybullet_data.getDataPath(),
        "plane100.urdf"))
startPosition=[0,0,0.3] # high enough that nothing touches the ground
startOrientationRPY = [0,0,0]
startOrientation = p.getQuaternionFromEuler(startOrientationRPY)   

designs = ['urdf/llllll.urdf', 'urdf/lwllwl.urdf']*4
for i_reset in range(5):

    robotID = p.loadURDF(
                 designs[i_reset], # ' misc_urdf/m6.urdf', 
                       basePosition=startPosition, baseOrientation=startOrientation,
                       flags= (p.URDF_MAINTAIN_LINK_ORDER 
                        | p.URDF_USE_SELF_COLLISION
                        | p.URDF_ENABLE_CACHED_GRAPHICS_SHAPES))
# 
    #                            | p.URDF_MERGE_FIXED_LINKS breaks things
#       | p.URDF_INITIALIZE_SAT_FEATURES  does not help
#  removing                         | p.URDF_USE_SELF_COLLISION does not help
#  | p.URDF_ENABLE_CACHED_GRAPHICS_SHAPES helps by allowing reload of same robot, but not with different robots

    # count all joints, including fixed ones
    num_joints_total = p.getNumJoints(robotID,
                    physicsClientId=physicsClient)

    # create lists of the joint names, types, and which ones are actuated
    moving_joint_names = []
    moving_joint_inds = []
    moving_joint_types = []
    moving_joint_limits = []
    moving_joint_centers = []
    for j_ind in range(num_joints_total):
        j_info = p.getJointInfo(robotID, 
            j_ind)
        if j_info[2] != (p.JOINT_FIXED):
            moving_joint_inds.append(j_ind)
            moving_joint_names.append(j_info[1])
            moving_joint_types.append(j_info[2])
            j_limits = [j_info[8], j_info[9]]
            j_center = (j_info[8] + j_info[9])/2
            if j_limits[1]<=j_limits[0]:
                j_limits = [-np.inf, np.inf]
                j_center = 0
            moving_joint_limits.append(j_limits)
            moving_joint_centers.append(j_center)

    # set joint states to a standing up pose
    num_joints = len(moving_joint_names)
    for i in range(num_joints):
        center = moving_joint_centers[i]
    #     if i in [2,5,8]:
    #         addition = -np.pi/2
    #     elif i in [11,14,17]:
    #         addition = np.pi/2
    #     else:
        addition = 0
        jind = moving_joint_inds[i]
        p.resetJointState( bodyUniqueId=robotID, 
            jointIndex = jind,
            targetValue= center+addition, 
            targetVelocity = 0 )

    # enable joint sensors
    for j in moving_joint_inds:
        p.enableJointForceTorqueSensor(robotID, j, 1)


    p.setJointMotorControlArray(
        bodyUniqueId=robotID, 
        jointIndices=moving_joint_inds, 
        controlMode=p.POSITION_CONTROL,
        targetPositions = moving_joint_centers,
        forces = np.ones(num_joints)*12
        )

    # steps to take with no control input before starting up
    # This drops to robot down to a physically possible resting starting position
    for i in range(100):

        p.stepSimulation()    
        joint_states = p.getJointStates(robotID,
                moving_joint_inds,
                physicsClientId=physicsClient)

        print('----------')
        print([j[3] for j in joint_states])
        # print out the joint moments too
    #     print([j[2][3] for j in joint_states])
    #     print([j[2][4] for j in joint_states])
    #     print([j[2][5] for j in joint_states])
        time.sleep(0.01)


    p.removeBody(robotID,
            physicsClientId=physicsClient)



----------
[2.3061520953935704e-17, 4.577379723450019e-16, 1.2177459812256214e-16, -4.369253103540694e-18, 6.05086426695383e-16, 0.0, 0.0, 0.0, 0.0, 0.0, 3.6399062624712644e-16, 1.136895776345944e-16, -2.168786547979802e-18, 3.485352126347885e-16, 1.0402533580160797e-17, 9.503772805891332e-18, 4.4151300140535373e-16, 2.314615200237915e-17]
----------
[-1.632250401864479e-17, 9.936230045563698e-17, -9.713929226183414e-17, -7.493181325745952e-18, 7.151310842210717e-18, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1299710110424046e-16, -5.338251907554544e-17, -6.48780837393095e-19, 2.5408109684846896e-16, -1.189036895556353e-17, -4.702594646444642e-18, 1.1650684312157047e-16, -2.052804486106748e-17]
----------
[5.158309661093885e-18, 2.3237711767105114e-16, 4.8042117308915376e-17, 5.170472866767439e-18, 2.8606513843640543e-16, 0.0, 0.0, 0.0, 0.0, 0.0, 1.9003446570970992e-16, -1.224974418119749e-18, 1.2755211236978394e-18, 1.8315785103144554e-16, -9.261124627015504e-18, -4.546290215715757e-19, 2.42860926446

----------
[1.440617381896637, -12.0, -3.9415578953558086, -0.0698333830478051, -12.0, 0.0, 0.0, 0.0, 0.0, 0.0, -12.0, -3.942805125267504, -0.06977884401568621, -12.0, -5.873401714843268, -2.063562382585215, -12.0, -3.9782794072223795]
----------
[1.3899783192330522, -12.0, -2.4833194820990747, 0.20278845487542313, -12.0, 0.0, 0.0, 0.0, 0.0, 0.0, -12.0, -2.4833573337545682, 0.20373700129742814, -12.0, -2.8560686808578204, -1.8759172075213195, -12.0, -2.5058547719672504]
----------
[1.3756888309931385, -12.0, -2.3234110821544087, 0.23301347329090996, -12.0, 0.0, 0.0, 0.0, 0.0, 0.0, -12.0, -2.3234029072062703, 0.23317644338495155, -12.0, -2.678142920442486, -1.8911616394697286, -12.0, -2.3544391812308216]
----------
[1.3568221306701647, -12.0, -2.164529492029624, 0.26260991765331654, -12.0, 0.0, 0.0, 0.0, 0.0, 0.0, -12.0, -2.1645168422974184, 0.26271405062915754, -12.0, -2.5011528956154203, -1.899326623993364, -12.0, -2.203788097585594]
----------
[1.3364617371576741, -12.0, -2.013443954

----------
[-0.15635012265849935, -0.6589066421003725, 0.22276641659472593, -0.057566632218760974, -0.6170300121385787, 0.0, 0.0, 0.0, 0.0, 0.0, -0.6352909796602407, 0.21732397430560058, -0.05711241431859256, -0.6425387278962168, 0.2388580680476529, 0.17932229991003062, -0.8208178361388072, 0.2650024527100572]
----------
[-0.20211104612086248, -1.4394146741191334, 0.4475038535896262, -0.06781902634078103, -1.2829008460972615, 0.0, 0.0, 0.0, 0.0, 0.0, -1.4300725687873364, 0.4454662005726241, -0.06701829682978061, -1.2929965901270544, 0.44003733983230475, 0.2615472977253031, -1.620014232791685, 0.49577354800232804]
----------
[-0.24159500586800145, -2.1263018323531653, 0.6403994383232103, -0.07891152019958486, -1.8686628278083344, 0.0, 0.0, 0.0, 0.0, 0.0, -2.1278960617056297, 0.6409898126379314, -0.07782828222428995, -1.8662799974587405, 0.6136468016430916, 0.334105289144867, -2.3194529218384363, 0.6940483858743051]
----------
[-0.27589484404559717, -2.731796677955795, 0.8074747860852196

----------
[1.5107949926959411e-18, 2.969095474019777e-16, 6.762328107701724e-18, 4.37851434692626e-18, -5.8256298533274055e-18, 0.0, 0.0, 0.0, 0.0, 0.0, -2.3741705746642468e-17, -6.804595051515244e-18, 2.548399696827344e-18, -2.1226496887427053e-17, 2.7636812493525935e-16, 2.0301159231733106e-18]
----------
[1.9225673652850232e-17, 1.2160972524249145e-16, -4.6050090619537495e-17, 7.952443069675144e-18, -1.203649736453819e-17, 0.0, 0.0, 0.0, 0.0, 0.0, -6.048647902016594e-17, -1.1005482205242814e-17, -3.6222057401195955e-18, -7.76900915263681e-18, 8.392717423809092e-17, -6.294444732917361e-17]
----------
[6.492470638042738e-18, 5.82540708840568e-16, 1.4554116767982482e-16, -7.880030675744654e-18, 1.867685827924061e-17, 0.0, 0.0, 0.0, 0.0, 0.0, 1.4293319463524004e-16, 3.1434650363151756e-18, 8.064025544338553e-18, 1.9816438234502082e-17, 6.996631749334873e-16, 1.733891923806997e-16]
----------
[1.2846639737747837e-17, 8.404876056726328e-16, 6.576419210225775e-17, 2.2342007659167752e-17, 

----------
[0.002833152405433595, 2.3867698772156127, 0.021913065871381854, 0.0013515377007940336, -0.002561159100860355, 0.0, 0.0, 0.0, 0.0, 0.0, 0.021908260330139376, 0.0020968479725200265, -0.002568509626511752, 0.002898333444577138, 2.3864021368509647, 0.021902343711937895]
----------
[0.002708184506031407, 2.3975197775962607, 0.019692038697596264, 0.0011921915841837524, -0.0023961577095389587, 0.0, 0.0, 0.0, 0.0, 0.0, 0.019697788862487037, 0.0018660742752886616, -0.0023121716080314455, 0.002745059597876615, 2.397467235945045, 0.01969994579931925]
----------
[0.0024425694799051164, 2.407154895812043, 0.017701175764577894, 0.0010177400088521886, -0.0022130561002225483, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01770291283339981, 0.0016696860517309198, -0.0020922701779198766, 0.0024698829437498006, 2.4071077763796285, 0.017708089726488522]
----------
[0.0021971189830013458, 2.415833478733952, 0.01590965745062183, 0.0008441730987738357, -0.0020283975401258324, 0.0, 0.0, 0.0, 0.0, 0.0, 0.015909922400

...

----------
[1.440617381896637, -12.0, -3.9415578953558086, -0.0698333830478051, -12.0, 0.0, 0.0, 0.0, 0.0, 0.0, -12.0, -3.942805125267504, -0.06977884401568621, -12.0, -5.873401714843268, -2.063562382585215, -12.0, -3.9782794072223795]
----------
[1.3899783192330522, -12.0, -2.4833194820990747, 0.20278845487542313, -12.0, 0.0, 0.0, 0.0, 0.0, 0.0, -12.0, -2.4833573337545682, 0.20373700129742814, -12.0, -2.8560686808578204, -1.8759172075213195, -12.0, -2.5058547719672504]
----------
[1.3756888309931385, -12.0, -2.3234110821544087, 0.23301347329090996, -12.0, 0.0, 0.0, 0.0, 0.0, 0.0, -12.0, -2.3234029072062703, 0.23317644338495155, -12.0, -2.678142920442486, -1.8911616394697286, -12.0, -2.3544391812308216]
----------
[1.3568221306701647, -12.0, -2.164529492029624, 0.26260991765331654, -12.0, 0.0, 0.0, 0.0, 0.0, 0.0, -12.0, -2.1645168422974184, 0.26271405062915754, -12.0, -2.5011528956154203, -1.899326623993364, -12.0, -2.203788097585594]
----------
[1.3364617371576741, -12.0, -2.013443954

----------
[-0.15635012265849935, -0.6589066421003725, 0.22276641659472593, -0.057566632218760974, -0.6170300121385787, 0.0, 0.0, 0.0, 0.0, 0.0, -0.6352909796602407, 0.21732397430560058, -0.05711241431859256, -0.6425387278962168, 0.2388580680476529, 0.17932229991003062, -0.8208178361388072, 0.2650024527100572]
----------
[-0.20211104612086248, -1.4394146741191334, 0.4475038535896262, -0.06781902634078103, -1.2829008460972615, 0.0, 0.0, 0.0, 0.0, 0.0, -1.4300725687873364, 0.4454662005726241, -0.06701829682978061, -1.2929965901270544, 0.44003733983230475, 0.2615472977253031, -1.620014232791685, 0.49577354800232804]
----------
[-0.24159500586800145, -2.1263018323531653, 0.6403994383232103, -0.07891152019958486, -1.8686628278083344, 0.0, 0.0, 0.0, 0.0, 0.0, -2.1278960617056297, 0.6409898126379314, -0.07782828222428995, -1.8662799974587405, 0.6136468016430916, 0.334105289144867, -2.3194529218384363, 0.6940483858743051]
----------
[-0.27589484404559717, -2.731796677955795, 0.8074747860852196

----------
[1.5107949926959411e-18, 2.969095474019777e-16, 6.762328107701724e-18, 4.37851434692626e-18, -5.8256298533274055e-18, 0.0, 0.0, 0.0, 0.0, 0.0, -2.3741705746642468e-17, -6.804595051515244e-18, 2.548399696827344e-18, -2.1226496887427053e-17, 2.7636812493525935e-16, 2.0301159231733106e-18]
----------
[1.9225673652850232e-17, 1.2160972524249145e-16, -4.6050090619537495e-17, 7.952443069675144e-18, -1.203649736453819e-17, 0.0, 0.0, 0.0, 0.0, 0.0, -6.048647902016594e-17, -1.1005482205242814e-17, -3.6222057401195955e-18, -7.76900915263681e-18, 8.392717423809092e-17, -6.294444732917361e-17]
----------
[6.492470638042738e-18, 5.82540708840568e-16, 1.4554116767982482e-16, -7.880030675744654e-18, 1.867685827924061e-17, 0.0, 0.0, 0.0, 0.0, 0.0, 1.4293319463524004e-16, 3.1434650363151756e-18, 8.064025544338553e-18, 1.9816438234502082e-17, 6.996631749334873e-16, 1.733891923806997e-16]
----------
[1.2846639737747837e-17, 8.404876056726328e-16, 6.576419210225775e-17, 2.2342007659167752e-17, 

----------
[0.002833152405433595, 2.3867698772156127, 0.021913065871381854, 0.0013515377007940336, -0.002561159100860355, 0.0, 0.0, 0.0, 0.0, 0.0, 0.021908260330139376, 0.0020968479725200265, -0.002568509626511752, 0.002898333444577138, 2.3864021368509647, 0.021902343711937895]
----------
[0.002708184506031407, 2.3975197775962607, 0.019692038697596264, 0.0011921915841837524, -0.0023961577095389587, 0.0, 0.0, 0.0, 0.0, 0.0, 0.019697788862487037, 0.0018660742752886616, -0.0023121716080314455, 0.002745059597876615, 2.397467235945045, 0.01969994579931925]
----------
[0.0024425694799051164, 2.407154895812043, 0.017701175764577894, 0.0010177400088521886, -0.0022130561002225483, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01770291283339981, 0.0016696860517309198, -0.0020922701779198766, 0.0024698829437498006, 2.4071077763796285, 0.017708089726488522]
----------
[0.0021971189830013458, 2.415833478733952, 0.01590965745062183, 0.0008441730987738357, -0.0020283975401258324, 0.0, 0.0, 0.0, 0.0, 0.0, 0.015909922400

----------
[2.3061520953935704e-17, 4.577379723450019e-16, 1.2177459812256214e-16, -4.369253103540694e-18, 6.05086426695383e-16, 0.0, 0.0, 0.0, 0.0, 0.0, 3.6399062624712644e-16, 1.136895776345944e-16, -2.168786547979802e-18, 3.485352126347885e-16, 1.0402533580160797e-17, 9.503772805891332e-18, 4.4151300140535373e-16, 2.314615200237915e-17]
----------
[-1.632250401864479e-17, 9.936230045563698e-17, -9.713929226183414e-17, -7.493181325745952e-18, 7.151310842210717e-18, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1299710110424046e-16, -5.338251907554544e-17, -6.48780837393095e-19, 2.5408109684846896e-16, -1.189036895556353e-17, -4.702594646444642e-18, 1.1650684312157047e-16, -2.052804486106748e-17]
----------
[5.158309661093885e-18, 2.3237711767105114e-16, 4.8042117308915376e-17, 5.170472866767439e-18, 2.8606513843640543e-16, 0.0, 0.0, 0.0, 0.0, 0.0, 1.9003446570970992e-16, -1.224974418119749e-18, 1.2755211236978394e-18, 1.8315785103144554e-16, -9.261124627015504e-18, -4.546290215715757e-19, 2.42860926446

In [ ]:
num_joints

In [ ]:
moving_joint_limits